In [1]:
import pandas as pd
import os
import re
import numpy as np
import pickle

pd.set_option('display.max_rows', None)

In [2]:
#다운로드 받은 파일 경로
download_file_path = r"/workspace/Gumi/Gumi_famous_restaurant/receipt_excel_data"
#Dataset볼때 개수제한 없애기
pd.set_option('display.max_rows', None)

#파일 목록 추출
file_list = sorted([download_file_path + "/" + f for f in os.listdir(download_file_path)],key=os.path.getctime)
excel_list = []

#파일 이름에서 부서명 추출하기 위한 정규표현식
pattern = r'\/([가-힣]+\d*[가-힣]+)\d*.xlsx$'

#파일 목록에서 파일을 읽고, 모든 열, 행이 Nan일때 제거, column 이름을 설정, 부서명 추가
for f in file_list:
    excel_file = pd.read_excel(f, header=None).dropna(how='all').dropna(axis = 1, how = 'all').iloc[1:].reset_index(drop=True)
    excel_file.columns = excel_file.iloc[0]
    excel_file = excel_file.iloc[1:]
    file_name = re.search(pattern, f).group(1)
    excel_file["부서명"] = file_name
    excel_list.append(excel_file)

In [3]:
#엑셀파일에서 제목열 형식이 중구난방이라 장소, 가격, 인원수 제목열 리스트를 직접 만들어 정리해줌
title_list = set()

for f in excel_list:
    title_list = title_list.union(f.columns.values)
    
title_list

{nan,
 '□ 도시환경국 환경보전과',
 '결재\n방법',
 '결재방법',
 '결제방법',
 '구      분',
 '구   분',
 '구  분',
 '구분',
 '금액(원)',
 '대상',
 '대상인원',
 '대상인원(명)',
 '대상자',
 '부  서  명',
 '부서',
 '부서명',
 '비  고\n(사용자)',
 '비고',
 '비고\n(사용자)',
 '비고(부서명)',
 '비고(사용자)',
 '비고(제공자)',
 '사   용   내   역',
 '사 용 처',
 '사용',
 '사용\n일자(일시)',
 '사용 일자',
 '사용 일자\n(일시)',
 '사용 일자 (일시)',
 '사용 일자(일시)',
 '사용 장소',
 '사용 장소\n(가맹점명)',
 '사용 장소(가맹점명)',
 '사용금액',
 '사용금액(원)',
 '사용내역',
 '사용대상',
 '사용목적',
 '사용목적(내역)',
 '사용목적(내역)\n* 사용대상 포함',
 '사용목적(내역)\n*사용대상 포함',
 '사용목적(내역) *사용대상 포함',
 '사용방법',
 '사용일시',
 '사용일자',
 '사용일자(일시)',
 '사용자',
 '사용장소',
 '사용장소\n(가맹점명)',
 '사용장소               (가맹점명)',
 '사용장소(가맹점명)',
 '사용처',
 '실지급액',
 '인원',
 '인원\n(명)',
 '인원(명)',
 '인원수',
 '일시',
 '지급명령내용',
 '지급액',
 '지급일자',
 '집행금액(원)',
 '집행대상',
 '집행대상자',
 '참석인원',
 '참석자',
 '참석자(명)',
 '통계목'}

In [4]:
#엑셀파일에서 제목열 형식이 중구난방이라 장소, 가격, 인원수 제목열 리스트를 만들어 정리해줌
restaurant_loc_index = ["사용처", "사용 장소\n(가맹점명)", "사용장소(가맹점명)", "사용장소", 
                        "사용장소\n(가맹점명)", "사용 장소(가맹점명)", "사 용 처", "사용장소               (가맹점명)",  "사용",
                        "사용 장소"]

restaurant_price_index = ["금액(원)", "사용금액(원)", "집행금액(원)", "지급액", "사용금액", "실지급액"]

restaurant_personnel_index = ["인원\n(명)", "대상인원(명)", "인원(명)", "참석인원", "대상인원", "참석자(명)", "인원", "참석자", 
                              "대상자",  "인원수"]

In [5]:
#구미 맛집정보 데이터프레임 생성
restaurant_data = pd.DataFrame()

restaurant_data["식당이름"] = np.nan
restaurant_data["가격"] = np.nan
restaurant_data["인원"] = np.nan
restaurant_data["부서명"] = np.nan

restaurant_data

,식당이름,가격,인원,부서명


In [6]:
#구미 맛집정보 데이터프레임에 값 추가해줌
for f in excel_list:
    
    #장소, 가격, 인원수 제목열에 해당하는 값을 찾아 변수에 저장해줌
    match_loc = [index for index in restaurant_loc_index if index in f]
    match_price = [index for index in restaurant_price_index if index in f]
    match_personnel = [index for index in restaurant_personnel_index if index in f]
    #엑셀 파일에 장소, 가격, 인원수 제목열이 다 들어있으면 구미 맛집정보 데이터프레임에 값 추가
    if(match_loc and match_price and match_personnel):
        temp = f[[match_loc[0], match_price[0],match_personnel[0],"부서명"]].rename(columns = {match_loc[0] : "식당이름", match_price[0] : "가격", match_personnel[0] : "인원"})
        
    restaurant_data = restaurant_data.append(temp)

#인덱서 초기화
restaurant_data = restaurant_data.reset_index(drop=True)
#인원수 열에서 (8, 8명, 직원 등 10명)형태로 되어있는 값들을 숫자만 추출해줌 
#바로 int64로 안바뀌는 버그가 있어서 float로 바꿔주고 다시 int64로 바꾸어주었음
restaurant_data.인원 = (restaurant_data.인원.astype(str).str.extract('(\d+)')).astype('float').astype('Int64')
restaurant_data

,식당이름,가격,인원,부서명
0,흑산도세꼬시식당,90000,6,신산업정책과
1,도미노피자남구미점,76590,10,공단2동
2,청정알탕 본점,58000,3,공단2동
3,청정알탕 본점,102000,8,공단2동
4,양파식당 송정직영점,150000,10,공단2동
5,큰집공탕서민밥상,72000,4,공단2동
6,실비식당,63000,9,공단2동
7,도미노피자남구미점,59440,2,공단2동
8,파리바게뜨 구미공단점,20560,2,공단2동
9,대백마트(공단점),385000,11,공단2동


In [7]:
restaurant_data = restaurant_data.dropna()
restaurant_data['식당이름'].replace(' ', '', regex=True, inplace=True)

restaurant_data

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,식당이름,가격,인원,부서명
0,흑산도세꼬시식당,90000,6,신산업정책과
1,도미노피자남구미점,76590,10,공단2동
2,청정알탕본점,58000,3,공단2동
3,청정알탕본점,102000,8,공단2동
4,양파식당송정직영점,150000,10,공단2동
5,큰집공탕서민밥상,72000,4,공단2동
6,실비식당,63000,9,공단2동
7,도미노피자남구미점,59440,2,공단2동
8,파리바게뜨구미공단점,20560,2,공단2동
9,대백마트(공단점),385000,11,공단2동


In [8]:
#구미_맛집_분석 데이터프레임을 만들고 [방문횟수, 사용금액, 인원, 인당가격, 부서]를 식당이름을 기준으로 묶어 정리해준다

restaurant_analysis = pd.DataFrame()
name_group = restaurant_data.groupby(['식당이름'])

restaurant_analysis["방문횟수"] = name_group['식당이름'].count()
restaurant_analysis["사용금액"] = name_group['가격'].sum()
restaurant_analysis["인원"] = name_group['인원'].sum()
restaurant_analysis["인당가격"] = restaurant_analysis['사용금액'] // restaurant_analysis['인원'] 
restaurant_analysis["부서"] = name_group['부서명'].unique()

restaurant_analysis.reset_index(inplace=True)

restaurant_analysis

,식당이름,방문횟수,사용금액,인원,인당가격,부서
0,(사)한국장애인환경실천협회,1,88000,28,3142,[회계과]
1,(주)실비아,2,682000,40,17050,[기획예산과]
2,(주)해피머니아이엔씨,3,180000,9,20000,[고아읍]
3,20막창,1,134000,10,13400,[고아읍]
4,3.5보리밥뷔페사곡점,2,221000,20,11050,[상모사곡동]
5,460식당,2,585000,25,23400,[복지정책과]
6,4공단돼지국밥생막창,1,104000,13,8000,[시립중앙도서관]
7,59쌀피자송정점,4,274500,61,4500,"[송정동, 아동보육과]"
8,7번가피자구미도량점외2,1,169000,19,8894,[차량등록사업소]
9,"BHC,파리바게트",1,147700,4,36925,[산동면]


In [9]:
with open('restaurant_analysis.pickle', 'wb') as f:
    pickle.dump(restaurant_analysis, f, pickle.HIGHEST_PROTOCOL)
